## !!!This notebook is not needed anymore, as you can directly write PNGs from MMF

In [ ]:
import os

import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import skimage.io
from tqdm import tqdm

In [ ]:
exp_name = 'realestate10k_nodepth_perceptual_l1laplacian_inpaintGonly_scale2_long'
n_iter = 100000
split = 'realestate10k_test'

synsin_list = '/private/home/ronghanghu/workspace/mmf_nr/mmf/datasets/builders/synsin_realestate10k/realestate_test.txt'
dump_dir = f"/private/home/ronghanghu/workspace/synsin/results_realestate10K_ours/{exp_name}/{n_iter}"
result_dir = f'/private/home/ronghanghu/workspace/mmf_nr/save/visualization_synsin_realestate10k/{exp_name}/{n_iter}/{split}'

In [ ]:
result_files = sorted(glob(os.path.join(result_dir, '*.npz')))

with open(synsin_list) as f:
    lines = f.readlines()
eval_image_id2idx = {
    '_'.join(l.split()[:3]).replace('/', '-'): n_l
    for n_l, l in enumerate(lines)
}
assert len(result_files) == len(eval_image_id2idx)


for file in tqdm(result_files):
    d = np.load(file)
    results = dict(d)
    d.close()

    base_image_id = os.path.basename(file).replace('_outputs.npz', '')
    image_id = '_'.join(base_image_id.split('_')[1:])
    idx = eval_image_id2idx[image_id]
    assert idx == int(base_image_id.split('_')[0])

    im_input = skimage.img_as_ubyte(results['orig_img_0'])
    im_output = skimage.img_as_ubyte(np.clip(results['rgb_1_inpaint'], 0, 1))
    im_tgt = skimage.img_as_ubyte(results['orig_img_1'])
    
    im_save_dir = os.path.join(dump_dir, f"{idx:04d}")
    os.makedirs(im_save_dir, exist_ok=True)

    skimage.io.imsave(im_save_dir + "/output_image_.png", im_output)
#     skimage.io.imsave(im_save_dir + "/input_image_.png", im_input)
#     skimage.io.imsave(im_save_dir + "/tgt_image_.png", im_tgt)

    # just for faster dumping -- copy input and tgt images, instead of writing them
    # only copy tgt_image_.png (input_image_.png is not needed for eval)
    from shutil import copy
    copy(f"/private/home/ronghanghu/workspace/synsin/results_realestate10K/{idx:04d}/tgt_image_.png",
         im_save_dir + "/tgt_image_.png")